<a href="https://colab.research.google.com/github/jsaulme/Datos_Masivos_2024/blob/main/DatosMasivos_Tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 2 - Operaciones con RDDS

In [1]:
!pip install pyspark
!pip install spotipy
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Segunda Clase de Spark").getOrCreate()
spark

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
import findspark
findspark.init()
findspark.find()
spark = SparkSession.builder.appName("Segunda Clase de Spark").getOrCreate()
spark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,42

In [2]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import base64

# Configuración de credenciales de Spotify desde variables de entorno o archivo seguro
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID", "d73dbd2a7d1248b8bc326eec8586a0e3")
# Assuming you have a string named 'YOUR_CLIENT_SECRET', replace this with your actual secret
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET", "YOUR_CLIENT_SECRET")

def get_access_token(client_id: str, client_secret: str) -> str:
    """Obtiene un token de acceso utilizando client_credentials."""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode(),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(url, headers=headers, data=data)
    if response.status_code != 200:
        error_msg = response.json().get("error_description", "No se pudo obtener el token de acceso")
        raise Exception(f"Error al obtener el token de acceso: {error_msg}")

    return response.json()["access_token"]

def create_spotify_client(access_token: str) -> spotipy.Spotify:
    """Crea una instancia del cliente de Spotify con el token de acceso."""
    credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=credentials_manager)
    return sp

In [3]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar las credenciales del archivo client_credential.py
import sys
sys.path.append('/content/drive/MyDrive')  # Añadir la ruta del archivo al sistema

from client_credential import CLIENT_SECRET  # Importar CLIENT_SECRET

# Configuración de credenciales de Spotify
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID", "d73dbd2a7d1248b8bc326eec8586a0e3")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Intentar recuperar información básica sobre un artista para verificar la conexión
try:
    # Obtener el token de acceso
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

    # Crear el cliente de Spotify usando el token de acceso
    spotify_client = create_spotify_client(access_token)

    # Buscar un artista para probar la conexión (por ejemplo, "Coldplay")
    result = spotify_client.search(q="Coldplay", type="artist", limit=1)

    # Imprimir el nombre y la popularidad del artista encontrado
    if result['artists']['items']:
        artist = result['artists']['items'][0]
        print(f"Conexión exitosa: Artista encontrado - {artist['name']}, Popularidad: {artist['popularity']}")
    else:
        print("Conexión exitosa, pero no se encontró el artista.")

except Exception as e:
    print(f"Error en la conexión: {e}")


Conexión exitosa: Artista encontrado - Coldplay, Popularidad: 92


In [5]:
import time
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configurar la autenticación con Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Buscar el artista "Intocable" para obtener su ID
artist_name = "Intocable"
results = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
artist = results['artists']['items'][0]
artist_id = artist['id']

# Información del artista
artist_info = {
    "Name": artist['name'],
    "Followers": artist['followers']['total'],
    "Popularity": artist['popularity'],
    "Genres": artist['genres']
}

print("Información del Artista:", artist_info)

# Obtener todos los álbumes del artista
albums = []
results = sp.artist_albums(artist_id, album_type='album', limit=50)
albums.extend(results['items'])

# Paginar álbumes si hay más de 50
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
    time.sleep(0.1)  # Pausa para no exceder límites

# Extraer información de cada álbum
album_data = []
for album in albums:
    album_info = {
        "AlbumID": album['id'],
        "AlbumName": album['name'],
        "ReleaseDate": album['release_date'],
        "TotalTracks": album['total_tracks'],
        "AlbumType": album['album_type']
    }
    album_data.append(album_info)

# Crear dataframe
df_albums = pd.DataFrame(album_data)
print(f"Total de álbumes: {len(df_albums)}")
print(df_albums)

# Obtener todas las pistas de cada álbum
track_data = []
for album in albums:
    album_id = album['id']
    results = sp.album_tracks(album_id)
    tracks = results['items']

    for track in tracks:
        track_info = {
            "TrackID": track['id'],
            "TrackName": track['name'],
            "AlbumID": album_id,
            "Duration_ms": track['duration_ms'],
            "Explicit": track['explicit'],
            "TrackNumber": track['track_number']
        }
        track_data.append(track_info)
    time.sleep(0.1)

# Crear dataframe
df_tracks = pd.DataFrame(track_data)
print(f"Total de pistas obtenidas: {len(df_tracks)}")
print(df_tracks)

# Obtener playlists donde aparece "Intocable"
playlists = []
results = sp.search(q=artist_name, type="playlist", limit=50)
playlists.extend(results['playlists']['items'])

# Paginación para obtener más playlists
while results['playlists']['next']:
    results = sp.next(results['playlists'])
    playlists.extend(results['playlists']['items'])
    time.sleep(0.1)  # Pausa para evitar límite

# Información de las playlists
playlist_data = []
for playlist in playlists:
    playlist_info = {
        "PlaylistID": playlist['id'],
        "PlaylistName": playlist['name'],
        "Owner": playlist['owner']['display_name'],
        "TotalTracks": playlist['tracks']['total']
    }
    playlist_data.append(playlist_info)

# Crear dataframe
df_playlists = pd.DataFrame(playlist_data)
print(f"Total de playlists encontradas: {len(df_playlists)}")
print(df_playlists)


Información del Artista: {'Name': 'Intocable', 'Followers': 3966116, 'Popularity': 73, 'Genres': ['banda', 'musica mexicana', 'norteno', 'tejano']}
Total de álbumes: 27
                   AlbumID                                          AlbumName  \
0   708g8olyAtie0YFU5gL1fQ                         Modus Operandi - Antología   
1   7zzc54afmjSxp7ozFkD8IJ                                     Modus Operandi   
2   1ubwEIz8qao13KXW7PpLdJ  Desde Casa, En Concierto, Y Canciones Desempol...   
3   6T90K6IjMkXv1JjRE8lXxB                                         Percepcíon   
4   1XdnsguIr95QPVg7lZ45rm                                            Highway   
5   6HWYb1m1FiMVQpvTRLi5Vr                           XX Aniversario (En Vivo)   
6   0nKSgzMBxxghnjJATsPm2Z                            En Peligro De Extinción   
7   105sAuhvY2L17r5DvIaTtf                                           Romances   
8   0iKVh2YiP0Ah83V4T2U1Lr                                               2011   
9   4fjmN8hFRaHWe8HEt

In [6]:
#Consolidar todos los dataframes en uno solo
df_final = pd.concat([df_albums, df_tracks, df_playlists], ignore_index=True)
df_final.head()
df_final.columns

Index(['AlbumID', 'AlbumName', 'ReleaseDate', 'TotalTracks', 'AlbumType',
       'TrackID', 'TrackName', 'Duration_ms', 'Explicit', 'TrackNumber',
       'PlaylistID', 'PlaylistName', 'Owner'],
      dtype='object')

In [7]:
# Ver los dataframes disponibles
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AlbumID       430 non-null    object 
 1   AlbumName     27 non-null     object 
 2   ReleaseDate   27 non-null     object 
 3   TotalTracks   1021 non-null   float64
 4   AlbumType     27 non-null     object 
 5   TrackID       403 non-null    object 
 6   TrackName     403 non-null    object 
 7   Duration_ms   403 non-null    float64
 8   Explicit      403 non-null    object 
 9   TrackNumber   403 non-null    float64
 10  PlaylistID    994 non-null    object 
 11  PlaylistName  994 non-null    object 
 12  Owner         994 non-null    object 
dtypes: float64(3), object(10)
memory usage: 144.8+ KB


In [8]:
artist_info.keys()

dict_keys(['Name', 'Followers', 'Popularity', 'Genres'])

In [9]:
print(df_tracks.columns)
print(df_albums.columns)
print(df_playlists.columns)

Index(['TrackID', 'TrackName', 'AlbumID', 'Duration_ms', 'Explicit',
       'TrackNumber'],
      dtype='object')
Index(['AlbumID', 'AlbumName', 'ReleaseDate', 'TotalTracks', 'AlbumType'], dtype='object')
Index(['PlaylistID', 'PlaylistName', 'Owner', 'TotalTracks'], dtype='object')


In [10]:
# Unión entre Tracks y Albums usando AlbumID
tracks_albums_df = df_tracks.merge(df_albums, on="AlbumID", how="inner")

# Visualizar el resultado
print(tracks_albums_df.head())


                  TrackID             TrackName                 AlbumID  \
0  0kZsra998ptD6T0TJbvuJX              Obsesión  708g8olyAtie0YFU5gL1fQ   
1  7ajRjzQCOtpPAOAJpt7LMf            Mi Castigo  708g8olyAtie0YFU5gL1fQ   
2  2lXPtOn8E8M9ru0t7ScY1L  Mi Alma Rota (Tanto)  708g8olyAtie0YFU5gL1fQ   
3  3F5H9yzLMoksgwLQqLndpD  El Amor Cuando Se Va  708g8olyAtie0YFU5gL1fQ   
4  1QpPqfNFXRWcyhNegZVkbF            Yo Seguiré  708g8olyAtie0YFU5gL1fQ   

   Duration_ms  Explicit  TrackNumber                   AlbumName ReleaseDate  \
0       152448     False            1  Modus Operandi - Antología  2024-10-24   
1       258620     False            2  Modus Operandi - Antología  2024-10-24   
2       194239     False            3  Modus Operandi - Antología  2024-10-24   
3       229579     False            4  Modus Operandi - Antología  2024-10-24   
4       253116     False            5  Modus Operandi - Antología  2024-10-24   

   TotalTracks AlbumType  
0           35     album  
1       

In [11]:
def create_relationship_table(sp, df_playlists):
    """Crea una tabla de relación entre TrackID y PlaylistID."""
    relationship_data = []

    for index, row in df_playlists.iterrows():
        playlist_id = row['PlaylistID']
        try:
            # Obtener las pistas de la lista de reproducción
            tracks = sp.playlist_tracks(playlist_id)
            for item in tracks['items']:
                track = item['track']
                if track:  # Asegurarse de que track no sea None
                    relationship_data.append({
                        'TrackID': track['id'],  # TrackID
                        'PlaylistID': playlist_id  # PlaylistID
                    })
        except Exception as e:
            print(f"Error al obtener pistas de la lista de reproducción {playlist_id}: {e}")

    # Crear un DataFrame a partir de la relación
    relationship_df = pd.DataFrame(relationship_data)
    return relationship_df

# Usar la función para crear la relación
relationship_df = create_relationship_table(sp, df_playlists)

# Visualizar la tabla de relación
#relationship_df.head


In [12]:
# Visualizar la tabla de relación
relationship_df.columns

Index(['TrackID', 'PlaylistID'], dtype='object')

In [13]:
import pandas as pd

# Paso 1: Unir df_tracks y relationship_df en base a TrackID
merged_df = pd.merge(relationship_df, df_tracks, on='TrackID', how='left')

# Paso 2: Unir el resultado anterior con df_albums en base a AlbumID
# Asegúrate de que df_tracks tenga una columna 'AlbumID'
merged_df = pd.merge(merged_df, df_albums, on='AlbumID', how='left')

# Paso 3: Unir el resultado anterior con df_playlists en base a PlaylistID
merged_df = pd.merge(merged_df, df_playlists, on='PlaylistID', how='left')

# Paso 4: Verifica el DataFrame consolidado
merged_df.columns
merged_df.head()


,TrackID,PlaylistID,TrackName,AlbumID,Duration_ms,Explicit,TrackNumber,AlbumName,ReleaseDate,TotalTracks_x,AlbumType,PlaylistName,Owner,TotalTracks_y
0,1OYoXMNCUYlpDyMONw2sNc,7mxnpmrHjToxcUki6X4sic,Alguien Te Va A Hacer Llorar,66JS64FD7dhGEKIHo2Uizr,249413.0,False,2.0,X,2005-01-01,15.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107
1,53IpyJOqUIcCzCUtMjP9oy,7mxnpmrHjToxcUki6X4sic,¿Y Todo Para Qué?,66rxjBaeraSB1qqQcYQSel,190533.0,False,8.0,Llévame Contigo,1996-01-01,12.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107
2,7A07Yu1dRND3RKDzbhKrDs,7mxnpmrHjToxcUki6X4sic,Fuerte No Soy,2UlbB3uHQNJlaYYBB2ljm0,267333.0,False,5.0,Contigo,1999-01-01,14.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107
3,2awJPHamaNQY8a1davj5k4,7mxnpmrHjToxcUki6X4sic,Amor Maldito,5Gr3x3xc7fYz6SALeFK7BH,204066.0,False,1.0,Intocable,1998-01-01,12.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107
4,3t4BZRyfrZGgICZGyNewyi,7mxnpmrHjToxcUki6X4sic,Eres Mi Droga,43UFBkcvSy10IIqsGIJzUw,207226.0,False,1.0,IV,1997-01-01,13.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107


In [14]:
print(df_tracks.columns)

Index(['TrackID', 'TrackName', 'AlbumID', 'Duration_ms', 'Explicit',
       'TrackNumber'],
      dtype='object')


In [15]:
print(merged_df.columns)

Index(['TrackID', 'PlaylistID', 'TrackName', 'AlbumID', 'Duration_ms',
       'Explicit', 'TrackNumber', 'AlbumName', 'ReleaseDate', 'TotalTracks_x',
       'AlbumType', 'PlaylistName', 'Owner', 'TotalTracks_y'],
      dtype='object')


In [16]:
# Realizar el merge entre df_tracks y merged_df en base a 'TrackID'
final_merged_df = pd.merge(merged_df, df_tracks, on='TrackID', how='left', suffixes=('', '_tracks'))

# Ahora, final_merged_df tendrá las columnas de ambos DataFrames
# Puedes optar por eliminar la columna duplicada de AlbumID o renombrarla si lo prefieres
final_merged_df.drop(columns=['AlbumID_tracks'], inplace=True, errors='ignore')  # Elimina la columna duplicada si es necesario

# Verifica el DataFrame final
#print(final_merged_df.info())
final_merged_df.head()

,TrackID,PlaylistID,TrackName,AlbumID,Duration_ms,Explicit,TrackNumber,AlbumName,ReleaseDate,TotalTracks_x,AlbumType,PlaylistName,Owner,TotalTracks_y,TrackName_tracks,Duration_ms_tracks,Explicit_tracks,TrackNumber_tracks
0,1OYoXMNCUYlpDyMONw2sNc,7mxnpmrHjToxcUki6X4sic,Alguien Te Va A Hacer Llorar,66JS64FD7dhGEKIHo2Uizr,249413.0,False,2.0,X,2005-01-01,15.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107,Alguien Te Va A Hacer Llorar,249413.0,False,2.0
1,53IpyJOqUIcCzCUtMjP9oy,7mxnpmrHjToxcUki6X4sic,¿Y Todo Para Qué?,66rxjBaeraSB1qqQcYQSel,190533.0,False,8.0,Llévame Contigo,1996-01-01,12.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107,¿Y Todo Para Qué?,190533.0,False,8.0
2,7A07Yu1dRND3RKDzbhKrDs,7mxnpmrHjToxcUki6X4sic,Fuerte No Soy,2UlbB3uHQNJlaYYBB2ljm0,267333.0,False,5.0,Contigo,1999-01-01,14.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107,Fuerte No Soy,267333.0,False,5.0
3,2awJPHamaNQY8a1davj5k4,7mxnpmrHjToxcUki6X4sic,Amor Maldito,5Gr3x3xc7fYz6SALeFK7BH,204066.0,False,1.0,Intocable,1998-01-01,12.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107,Amor Maldito,204066.0,False,1.0
4,3t4BZRyfrZGgICZGyNewyi,7mxnpmrHjToxcUki6X4sic,Eres Mi Droga,43UFBkcvSy10IIqsGIJzUw,207226.0,False,1.0,IV,1997-01-01,13.0,album,Intocable - Lo Mejor de Intocable! 🤠🪗,José Eduardo Serrano,107,Eres Mi Droga,207226.0,False,1.0


In [17]:
final_merged_df.columns

Index(['TrackID', 'PlaylistID', 'TrackName', 'AlbumID', 'Duration_ms',
       'Explicit', 'TrackNumber', 'AlbumName', 'ReleaseDate', 'TotalTracks_x',
       'AlbumType', 'PlaylistName', 'Owner', 'TotalTracks_y',
       'TrackName_tracks', 'Duration_ms_tracks', 'Explicit_tracks',
       'TrackNumber_tracks'],
      dtype='object')

In [18]:
#Renombrar las columnas
# Definir un diccionario de mapeo para los nuevos nombres de las columnas
new_column_names = {
    'TrackID': 'track_id',
    'PlaylistID': 'playlist_id',
    'TrackName': 'track_name',
    'AlbumID': 'album_id',
    'Duration_ms': 'duration_ms',
    'Explicit': 'explicit',
    'TrackNumber': 'track_number',
    'AlbumName': 'album_name',
    'ReleaseDate': 'release_date',
    'TotalTracks_x': 'total_tracks_album',
    'AlbumType': 'album_type',
    'PlaylistName': 'playlist_name',
    'Owner': 'playlist_owner',
    'TotalTracks_y': 'total_tracks_playlist',
    'TrackName_tracks': 'track_name_tracks',
    'Duration_ms_tracks': 'duration_ms_tracks',
    'Explicit_tracks': 'explicit_tracks',
    'TrackNumber_tracks': 'track_number_tracks'
}

# Renombrar las columnas del DataFrame final
final_merged_df = final_merged_df.rename(columns=new_column_names)

# Mostrar las columnas renombradas para confirmar
print(final_merged_df.columns)


Index(['track_id', 'playlist_id', 'track_name', 'album_id', 'duration_ms',
       'explicit', 'track_number', 'album_name', 'release_date',
       'total_tracks_album', 'album_type', 'playlist_name', 'playlist_owner',
       'total_tracks_playlist', 'track_name_tracks', 'duration_ms_tracks',
       'explicit_tracks', 'track_number_tracks'],
      dtype='object')


In [19]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Before converting to PySpark DataFrame, enforce data type consistency
for col in final_merged_df.columns:
    # Check if the column has mixed types, and if so, convert to string
    if final_merged_df[col].apply(type).nunique() > 1:
        final_merged_df[col] = final_merged_df[col].astype(str)

# Convert the Pandas DataFrame to a PySpark DataFrame
final_merged_spark_df = spark.createDataFrame(final_merged_df)

# Now you can access the RDD
final_merged_rdd = final_merged_spark_df.rdd

# Display partition count and sample rows
print(f"Number of partitions: {final_merged_rdd.getNumPartitions()}")
print(final_merged_rdd.take(5))

Number of partitions: 2
[Row(track_id='1OYoXMNCUYlpDyMONw2sNc', playlist_id='7mxnpmrHjToxcUki6X4sic', track_name='Alguien Te Va A Hacer Llorar', album_id='66JS64FD7dhGEKIHo2Uizr', duration_ms=249413.0, explicit='False', track_number=2.0, album_name='X', release_date='2005-01-01', total_tracks_album=15.0, album_type='album', playlist_name='Intocable - Lo Mejor de Intocable! 🤠🪗', playlist_owner='José Eduardo Serrano', total_tracks_playlist=107, track_name_tracks='Alguien Te Va A Hacer Llorar', duration_ms_tracks=249413.0, explicit_tracks='False', track_number_tracks=2.0), Row(track_id='53IpyJOqUIcCzCUtMjP9oy', playlist_id='7mxnpmrHjToxcUki6X4sic', track_name='¿Y Todo Para Qué?', album_id='66rxjBaeraSB1qqQcYQSel', duration_ms=190533.0, explicit='False', track_number=8.0, album_name='Llévame Contigo', release_date='1996-01-01', total_tracks_album=12.0, album_type='album', playlist_name='Intocable - Lo Mejor de Intocable! 🤠🪗', playlist_owner='José Eduardo Serrano', total_tracks_playlist=107

# Tarea 2

In [20]:
# Instala PySpark y la biblioteca de datasets de Hugging Face
!pip install pyspark
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00


In [22]:
# Importar las bibliotecas necesarias
import pandas as pd
from datasets import load_dataset

# Cargar el dataset de Hugging Face
dataset_name = "dazzle-nu/CIS435-CreditCardFraudDetection"
dataset = load_dataset(dataset_name)

# Verificar el contenido del dataset
print(dataset)  # Información general del dataset
print(dataset['train'].column_names)  # Nombres de las columnas en la partición de entrenamiento
print(dataset['train'][0])  # Mostrar el primer registro

fraudTrain.csv:   0%|          | 0.00/267M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1048575 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'Unnamed: 23', '6006'],
        num_rows: 1048575
    })
})
['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'Unnamed: 23', '6006']
{'Unnamed: 0': 0, 'trans_date_trans_time': '1/1/19 0:00', 'cc_num': 2703190000000000.0, 'merchant': 'fraud_Rippin, Kub and Mann', 'category': 'misc_net', 'amt': 4.97, 'first': 'Jennifer', 'last': 'Banks', 'gender': 'F', 'street': '561 Perry Cove', 'city': 'Moravian Falls', 'state': 'NC', 'zip': 28654, 'lat': 36.0788, 'long': -81.1781, 'city_pop': 3495, 'job': 'Psych

In [24]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from datasets import load_dataset

# Inicializar la sesión de Spark
spark = SparkSession.builder \
    .appName("HuggingFace to RDD") \
    .getOrCreate()

# Convertir el dataset a un DataFrame de PySpark directamente
spark_df = spark.createDataFrame(dataset['train'])

# Mostrar las primeras filas del DataFrame
spark_df.show()


PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.

In [26]:
# Access the 'train' split of the dataset to get the data types.
print(dataset['train'].features)

{'Unnamed: 0': Value(dtype='int64', id=None), 'trans_date_trans_time': Value(dtype='string', id=None), 'cc_num': Value(dtype='float64', id=None), 'merchant': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'amt': Value(dtype='float64', id=None), 'first': Value(dtype='string', id=None), 'last': Value(dtype='string', id=None), 'gender': Value(dtype='string', id=None), 'street': Value(dtype='string', id=None), 'city': Value(dtype='string', id=None), 'state': Value(dtype='string', id=None), 'zip': Value(dtype='int64', id=None), 'lat': Value(dtype='float64', id=None), 'long': Value(dtype='float64', id=None), 'city_pop': Value(dtype='int64', id=None), 'job': Value(dtype='string', id=None), 'dob': Value(dtype='string', id=None), 'trans_num': Value(dtype='string', id=None), 'unix_time': Value(dtype='int64', id=None), 'merch_lat': Value(dtype='float64', id=None), 'merch_long': Value(dtype='float64', id=None), 'is_fraud': Value(dtype='int64', id=None), 'Unnamed: 23': 

In [28]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.types import * # Import all types using *
# or
# from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType # Import individual types
from datasets import load_dataset

# Define the explicit schema according to the types you have shared
schema = StructType([
    StructField("Unnamed: 0", IntegerType(), True),
    StructField("trans_date_trans_time", StringType(), True),
    StructField("cc_num", DoubleType(), True),
    StructField("merchant", StringType(), True),
    StructField("category", StringType(), True),
    StructField("amt", DoubleType(), True),
    StructField("first", StringType(), True),
    StructField("last", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("zip", IntegerType(), True),
    StructField("lat", DoubleType(), True),
    StructField("long", DoubleType(), True),
    StructField("city_pop", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("trans_num", StringType(), True),
    StructField("unix_time", IntegerType(), True),
    StructField("merch_lat", DoubleType(), True),
    StructField("merch_long", DoubleType(), True),
    StructField("is_fraud", IntegerType(), True),
    StructField("Unnamed: 23", DoubleType(), True),
    StructField("6006", DoubleType(), True)
])

# Convert the Pandas DataFrame to a PySpark DataFrame using the defined schema
# spark_df = spark.createDataFrame(dataset, schema=schema) # This line was causing error
spark_df = spark.createDataFrame(dataset['train'], schema=schema) # Changed dataset to dataset['train']


# Verify the schema and content
spark_df.printSchema()
spark_df.show(5)

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- cc_num: double (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)
 |-- Unnamed: 23: double (nullable = true)
 |-- 6006: double (nullable = true)

+----------+-------

# Estadísticas descriptivas

## Monto de transacción por categoría.
## Calcula el monto promedio, máximo y mínimo de las transacciones:

In [30]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F # Import and alias the functions module

spark_df.groupBy("category").agg(
    F.avg("amt").alias("average_amt"),
    F.max("amt").alias("max_amt"),
    F.min("amt").alias("min_amt"),
    F.stddev("amt").alias("stddev_amt")
).show()

+--------------+------------------+-------+-------+------------------+
|      category|       average_amt|max_amt|min_amt|        stddev_amt|
+--------------+------------------+-------+-------+------------------+
|        travel|111.24835851355483|28948.9|    1.0| 611.2931016905442|
|      misc_net| 81.10567146940242|4084.34|    1.0| 167.8910932345687|
|   grocery_pos|116.98304225972431| 397.97|  10.87| 53.20511259611615|
|     kids_pets| 57.50786409785187| 586.34|    1.0|  48.7173079513711|
|  shopping_net| 88.46497344706482|9555.26|    1.0|249.52796900498944|
|   grocery_net|53.740091778098574| 185.81|   1.49|22.492844099215727|
|   food_dining| 51.12934293161918| 769.26|    1.0|48.911455155915455|
| gas_transport|63.421289673964765| 154.03|   5.93|15.876772861755272|
| personal_care| 47.98750986421418| 527.03|    1.0| 49.28049734629429|
|health_fitness| 54.13251045240951| 598.38|    1.0| 48.47519273752997|
| entertainment| 64.16067306300289| 934.47|    1.0| 66.15704199855568|
|     

# Edad aproximada de los usuarios al momento de la transacción

## Calcula la edad en años basada en la columna de fecha de nacimiento (dob) y el timestamp de la transacción (unix_time). Esto proporciona la edad estimada de cada persona en cada transacción:

In [31]:
from pyspark.sql.functions import col, from_unixtime, datediff, year

# Convierte unix_time a fecha y calcula edad
spark_df = spark_df.withColumn("transaction_date", from_unixtime(col("unix_time")))
spark_df = spark_df.withColumn("dob", F.to_date("dob", "yyyy-MM-dd"))
spark_df = spark_df.withColumn("age", (year("transaction_date") - year("dob")))

# Edad promedio, mínima y máxima por género
spark_df.groupBy("gender").agg(
    F.avg("age").alias("average_age"),
    F.max("age").alias("max_age"),
    F.min("age").alias("min_age")
).show()


+------+-----------+-------+-------+
|gender|average_age|max_age|min_age|
+------+-----------+-------+-------+
|     F|       NULL|   NULL|   NULL|
|     M|       NULL|   NULL|   NULL|
+------+-----------+-------+-------+



# Análisis de transacciones por género y ocupación
## Analiza el monto promedio y la probabilidad de fraude basado en género y ocupación:

In [ ]:
# # Monto promedio y probabilidad de fraude por género y ocupación
# occupation_fraud_analysis = spark_df.groupBy("gender", "job").agg(
#     F.avg("amt").alias("average_amt"),
#     (F.sum("is_fraud") / F.count("is_fraud") * 100).alias("fraud_percentage")
# ).orderBy("fraud_percentage", ascending=False)

# occupation_fraud_analysis.show()
